In [86]:
# Set in root_directory
%cd /home/ronald/PycharmProjects/x-ray-deep-learning/X-ray_Object_Detection/
#%ls    
# libs 
import numpy as np    
from pathlib import Path
import json

np.random.seed(1)

# Directories
ROOT = Path('data/raw')
images_path = ROOT / 'images'
ann_path = ROOT/ 'annotation'
print('Images Path:', images_path)
print('Annotation Path:', ann_path)

# Labels/n_classes
labels = ['gun'] #, 'knife', 'shuriken', 'razor_blade']
n_classes = len(labels) + 1 # count background

# Image Dimenssions
dim = (256, 256, 1)

# Collect all files absolute Path 
imgs_paths = sorted([i.absolute() for i in images_path.glob("*.png") if i.is_file()])

indexes = np.arange(len(imgs_paths))

batch_size = 4
index = 15
# Set batch indexes
# if index 0 and batch 4 in range(0, 17) retrieve values [0 1 2 3]
# if index 1 and batch 4 in range(0, 17) retrieve values [4 5 6 7]
#indexes = indexes[index * batch_size:(index + 1) * batch_size]

#imgs_paths = [imgs_paths[index] for index in indexes]
imgs_name = [img.name for img in imgs_paths]

# Create empty data-set.
X = np.empty((batch_size, *dim), dtype=np.float32)
y = np.empty((batch_size, dim[0], dim[1], n_classes), dtype=np.float32)

/home/ronald/PycharmProjects/x-ray-deep-learning/X-ray_Object_Detection
Images Path: data/raw/images
Annotation Path: data/raw/annotation


In [87]:
# Open imgs annotations
with open("data/raw/annotation/coco_annotation.json", "r") as read_it: 
     ann_data = json.load(read_it)
        


In [88]:
import cv2
import numpy as np
import imgaug as ia
import imgaug.augmenters as iaa
from imgaug.augmentables import Keypoint, KeypointsOnImage
%matplotlib inline
from matplotlib import pyplot as plt
dict_imgs = ann_data.get('images')
dict_ann = ann_data.get('annotations')
dict_cat = ann_data.get('categories')

seq = iaa.Sequential([
    iaa.Fliplr(0.5),# horizontal flips
    # Small gaussian blur with random sigma between 0 and 0.5.
    iaa.GaussianBlur(sigma=(0, 0.5)), 
    # Crop image with random from 0 to 10%    
    # But we only crop about 50% of all images.
    iaa.Sometimes(
        0.5,
       iaa.Crop(percent=(0, 0.1), keep_size=True)),
    # Strengthen or weaken the contrast in each images.
    iaa.LinearContrast((0.75, 1)),

    # Add gaussian noise.
    # For 30% of all images, we sample the noise once per pixel.
    # For the other 30% of all images, we sample the noise per pixel AND
    # channel. This can change the color (not only brightness) of the
    # pixels.
    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05), per_channel=0.3),

    # Apply affine transformations to each images.
    # Scale/zoom them.
    iaa.Affine(
        scale={"x": (1.0, 1.1), "y": (1.0, 1.1)})
], random_order=True) # apply augmenters in random order
 

In [90]:
def search_array(array, key, value):
    return next((obj for obj in array if obj[key] == value), None) # return object

def get_img_seg_kps(img_seg):
    points = list()
    for i in range(0, len(img_seg), 2): # iterate every two steps
        chunk = img_seg[i:i+2]
        points.append(Keypoint(x=chunk[0], y=chunk[1]))
    
    return points

def get_img_info(img_name):
    """
    return img_label and segmentation points of the image
    """
    img_seg, label = None, None
    img_obj = search_array(dict_imgs, 'file_name', img_name)
    if img_obj is not None:
        ann_obj = search_array(dict_ann, 'image_id', str(img_obj['id']))
        if ann_obj is not None:
            kps = get_img_seg_kps(ann_obj['segmentation'])
            label = search_array(dict_cat, 'id', ann_obj['category_id'])
            return label['name'], kps
        else: # Create annotation for image
            kps = create_img_seg(img_obj)
            return 'background', kps
    
    return None

def create_img_seg(img_obj):
    height = img_obj['height']
    width = img_obj['width']
    points = [
        Keypoint(x=0, y=0),
        Keypoint(x=width-1, y=0),
        Keypoint(x=width-1, y=height-1),
        Keypoint(x=0, y=height-1)
    ]
#    print(points)
    return points

def get_augimg(img, img_info):
    label, points = img_info
    kps = KeypointsOnImage(points, shape=img.shape)
    if img.shape != dim:
        img = ia.imresize_single_image(img, dim[0:2])
        kps = kps.on(img)
    # Augment keypoints and images.
    seq_det = seq.to_deterministic()
    img_aug = seq_det.augment_images([img])[0]
    kps_aug = seq_det.augment_keypoints([kps])[0]
#     print(kps)
#     print("--------\n", kps_aug)
#     img_aug, kps_aug = seq(image=img, keypoints=kps)
    aug_points = [[kp.x, kp.y] for kp in kps_aug.keypoints]
    aug_points_dic = {'label': label, 'points': aug_points}
#     ia.imshow(np.hstack([
#         kps.draw_on_image(img, size=10),
#         kps_aug.draw_on_image(img_aug, size=10)]))

    return img_aug, aug_points_dic
    
def show(img):
    print(img.shape)
    plt.imshow(img)
    plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
    plt.show()

def get_mask(img, imgaug_shape):
    blank = np.zeros(shape=(img.shape[0], img.shape[1]), dtype=np.float32)
    points = np.array(imgaug_shape['points'], dtype=np.int32)
    label = imgaug_shape['label']
    cv2.fillPoly(blank, [points], 255)
    blank = blank / 255.0
#     ia.imshow(img)
#     ia.imshow(blank)
    return np.expand_dims(blank, axis=2)

    
def data_generation(img_path):
    X = np.empty((batch_size, *dim), dtype=np.float32)
    y = np.empty((batch_size, dim[0], dim[1], n_classes), dtype=np.float32)    
    
    # retrieve img in gray_scale as numpy
    img = cv2.imread(str(img_path), 0) # our images are gray_scale
    img = np.expand_dims(img, axis=2)
#     img = (img / 255.0).astype(np.float32)
    images = [np.copy(img) for _ in range(batch_size)]
    img_info = get_img_info(img_path.name)
    for i, image in enumerate(images):
        imgaug, imgaug_shape = get_augimg(img, img_info)
        imgaug_mask = get_mask(imgaug, imgaug_shape)
        print(imgaug.shape)
        print(imgaug_mask.shape)
        X[i,] = imgaug
        y[i,] = imgaug_mask
    
    return X, y
    
img_pol = data_generation(imgs_paths[index])
#print(img_pol)

(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
